<a href="https://colab.research.google.com/github/AlvaroMartins7/misc/blob/main/imdb_alvaromartins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch --quiet
!pip install --upgrade datasets huggingface_hub --quiet
!pip install fsspec==2023.6.0 --quiet

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from google.colab import userdata

# SETAR A CHAVE DO HUGGINGFACE
login(token = userdata.get('HF_TOKEN'))

# dataset IMDb
dataset = load_dataset("imdb")

In [ ]:
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis")

In [ ]:
# Pega 5 exemplos de teste
test_texts = dataset["test"]["text"][:5]

# Faz a análise
results = sentiment_analyzer(test_texts)

for text, result in zip(test_texts, results):
    print(f"Texto: {text[:250]}...")
    print(f"Sentimento: {result['label']}, score: {result['score']:.4f}")
    print()


In [ ]:
# previsão
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

max_length = 512

test_texts_short = [text[:max_length] for text in test_texts]
results = sentiment_analyzer(test_texts_short, batch_size=32)

# Traduz rótulos de comparação
def label_to_int(label):
    return 1 if label == "POSITIVE" else 0

pred_labels = [label_to_int(r["label"]) for r in results]

# acurácia
correct = sum([p == t for p, t in zip(pred_labels, test_labels)])
accuracy = correct / len(test_labels)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")
